# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from tqdm import tqdm
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [2]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

0it [00:00, ?it/s]Downloading http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz to ./mnist/MNIST/raw/train-images-idx3-ubyte.gz
100%|█████████▉| 9912320/9912422 [09:23<00:00, 22184.95it/s]Extracting ./mnist/MNIST/raw/train-images-idx3-ubyte.gz to ./mnist/MNIST/raw

0it [00:00, ?it/s]Downloading http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz to ./mnist/MNIST/raw/train-labels-idx1-ubyte.gz

 28%|██▊       | 8192/28881 [00:00<00:00, 32145.49it/s]
32768it [00:01, 31681.50it/s]

0it [00:00, ?it/s]Extracting ./mnist/MNIST/raw/train-labels-idx1-ubyte.gz to ./mnist/MNIST/raw

 30%|███       | 499712/1648877 [00:14<00:46, 24715.24it/s]
9920512it [09:40, 22184.95it/s]
100%|█████████▉| 1646592/1648877 [01:04<00:00, 24072.31it/s]

0it [00:00, ?it/s]Extracting ./mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to ./mnist/MNIST/raw


  0%|          | 0/4542 [00:00<?, ?it/s]

8192it [00:00, 25237.79it/s]Extracting ./mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./mnist/MNIST/raw
Proces

Then, we define the model, object function and optimizer that we use to classify.

In [3]:
class SimpleNet(nn.Module):
# TODO:define model
    def __init__(self):
        super().__init__()
        # 1,28x28
        self.conv1= nn.Conv2d(1,10,5) # 10, 24x24
        self.conv2=nn.Conv2d(10,20,3) # 128, 10x10
        self.fc1 = nn.Linear(20*10*10,500)
        self.fc2 = nn.Linear(500,10)
    def forward(self,x):
        in_size = x.size(0)
        
        out = self.conv1(x) #24
        out = F.relu(out)
        out = F.max_pool2d(out, 2, 2)  #12
        
        out = self.conv2(out) #10
        out = F.relu(out)
        out = out.view(in_size,-1)
        
        out = self.fc1(out)
        out = F.relu(out)
        
        out = self.fc2(out)
        out = F.log_softmax(out,dim=1)
        return out


    
model = SimpleNet()
# TODO:define loss function and optimiter
criterion = torch.nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters())

Next, we can start to train and evaluate!

In [6]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(DEVICE)
# train and evaluate

for epoch in range(1,1+NUM_EPOCHS):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
    #         print(output.shape, target.shape)
    #         print(output.dtype, target.dtype)
    #         print('\n\n')
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if(batch_idx+1)%30 == 0: 
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

    # Evaluate the training error
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in train_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # 将一批的损失相加
            pred = output.max(1, keepdim=True)[1] # 找到概率最大的下标
            correct += pred.eq(target.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    print('\nTraining set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(train_loader.dataset),
            100. * correct / len(train_loader.dataset)))

    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # 将一批的损失相加
            pred = output.max(1, keepdim=True)[1] # 找到概率最大的下标
            correct += pred.eq(target.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset),
            100. * correct / len(test_loader.dataset)))
        
    
    


Train Epoch: 1 [3712/60000 (6%)]	Loss: 0.002428
Train Epoch: 1 [7552/60000 (13%)]	Loss: 0.022574
Train Epoch: 1 [11392/60000 (19%)]	Loss: 0.001869
Train Epoch: 1 [15232/60000 (25%)]	Loss: 0.000149
Train Epoch: 1 [19072/60000 (32%)]	Loss: 0.000577
Train Epoch: 1 [22912/60000 (38%)]	Loss: 0.001686
Train Epoch: 1 [26752/60000 (45%)]	Loss: 0.000652
Train Epoch: 1 [30592/60000 (51%)]	Loss: 0.000157
Train Epoch: 1 [34432/60000 (57%)]	Loss: 0.000412
Train Epoch: 1 [38272/60000 (64%)]	Loss: 0.000671
Train Epoch: 1 [42112/60000 (70%)]	Loss: 0.002315
Train Epoch: 1 [45952/60000 (77%)]	Loss: 0.000966
Train Epoch: 1 [49792/60000 (83%)]	Loss: 0.000071
Train Epoch: 1 [53632/60000 (90%)]	Loss: 0.002035
Train Epoch: 1 [57472/60000 (96%)]	Loss: 0.000264

Training set: Average loss: 0.0323, Accuracy: 59790/60000 (100%)


Test set: Average loss: 0.0386, Accuracy: 9881/10000 (99%)

Train Epoch: 2 [3712/60000 (6%)]	Loss: 0.002242
Train Epoch: 2 [7552/60000 (13%)]	Loss: 0.040883
Train Epoch: 2 [11392/60000 

#### Q5:
Please print the training and testing accuracy.

Training Set Accuracy = 99.6% 
Testing Set Accuracy = 99.03%